# LDA dataset collection
John Rodriguez, 2020-03-01

In [1]:
logan_path = '../../../'
import sys
sys.path.append(logan_path)
from parlogan.obs import OBs_between
from parlogan.systems import related_system_errors
from parlogan.db.es import EmptyElasticQuery
from parlogan.color import paranal
import pandas as pd

In [2]:
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LdaMulticore
from sklearn.model_selection import train_test_split
from numpy.random import seed
seed(2020)

## Getting the OBs

In [4]:
#2020-02-05T09:29:42.398
dates = ["2020-02-01T02:41:00.957", "2020-01-30T07:42:08.717", "2020-01-28T07:22:10.398",
         "2020-01-27T06:30:37.264", "2019-07-03T00:52:00.233", "2020-02-05T09:29:42.398"
         "2020-01-30T07:15:56.888", "2019-11-10T01:06:03.160", "2019-09-06T07:50:23.224",
         "2019-08-17T03:14:13.765", "2019-07-14T00:57:01.058", "2019-07-05T23:50:04.762",
         "2019-07-05T23:50:37.027", "2019-06-09T00:31:33.953 ", "2019-06-03T23:53:01.879",
         "2019-04-22T23:34:09.339"
        ]

granularity = '5 min' #None for no granularity
system = 'GRAVITY'

for date in dates:
    date = pd.to_datetime(date, utc = True)
    system = 'GRAVITY'
    start = date - pd.Timedelta('5 hour')
    end = date + pd.Timedelta('5 hour')
    OB = OBs_between(system, start, end)
    OB = OB[(OB['START'] < date) & (OB['END'] > date)]
    OB_start, OB_end = OB[['START', 'END']].iloc[0]
    errors = related_system_errors(OB_start, OB_end, system, delta = granularity)
    
    all_errors = []
    for system_groups in errors.values():
        for system_errors in system_groups.values():
            all_errors.append(system_errors)
    
    if granularity != None:
        all_errors = pd.concat(all_errors).sort_values(by = "@timestamp").reset_index(drop = True)
        all_errors = granularity
    else:
        all_errors['@timestamp'] = OB_start
        all_errors['granularity'] = f"{(OB_end - OB_start).total_seconds / 60 % 60} min"
    display(all_errors)
    break

,@timestamp,errkey,system,count
0,2020-02-01 02:32:00+00:00,evhERR_CMD_NOT_CHECKED,ISS,2
1,2020-02-01 02:32:00+00:00,eccsERR_BUF_TOO_BIG,AT2,3
2,2020-02-01 02:32:00+00:00,eccsERR_BUF_TOO_BIG,AT3,3
3,2020-02-01 02:32:00+00:00,eccsERR_BUF_TOO_BIG,AT4,3
4,2020-02-01 02:32:00+00:00,eccsERR_MSG,NAOMI1,15
5,2020-02-01 02:32:00+00:00,ccsERR_MSG_Q_EMPTY,NAOMI1,15
6,2020-02-01 02:32:00+00:00,eccsERR_MSG,NAOMI2,6
7,2020-02-01 02:32:00+00:00,eccsERR_BUF_TOO_BIG,AT1,3
8,2020-02-01 02:32:00+00:00,ccsERR_MSG_Q_EMPTY,NAOMI2,6
9,2020-02-01 02:32:00+00:00,eccsERR_MSG,NAOMI4,6
